In [1]:
import torch
import onnx
import onnx_tf.backend
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

C:\Users\potap\FinARL\venv_10\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.config import INDICATORS, TRAINED_MODEL_DIR
from finrl.main import check_and_make_directories
from Envs.env_stocktrading import StockTradingEnv

check_and_make_directories([TRAINED_MODEL_DIR])

## Read data

We first read the .csv file of our training data into dataframe.

In [31]:
train = pd.read_csv(r"C:\Users\potap\OneDrive\Desktop\hourly\hourly_training_data.csv")

# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following two lines.
def prep(train):
    # Sort the DataFrame by the 'date' column
    train = train.sort_values(by='date')

    # Create a mapping of unique dates to their corresponding index
    date_mapping = {date: idx for idx, date in enumerate(train['date'].unique())}

    # Map the 'date' column to the date_mapping to get the new index
    train['index'] = train['date'].map(date_mapping)

    # Set the new index for the DataFrame
    train.set_index('index', inplace=True, drop=True)
    train = train.fillna(0)
    return train

train = prep(train)

In [32]:
train

,date,open,high,low,close,volume,tic,day,macd,boll_ub,...,close_30_sma,close_60_sma,turbulence,vix,stocktwitsPosts,stocktwitsLikes,stocktwitsImpressions,stocktwitsSentiment,random,change
index,,,,,,,,,,,,,,,,,,,,,
0,2021-11-24 14:00:00,159.380,159.703,158.862,159.085,3513469.0,AAPL,5,0.000000,160.308182,...,159.085000,159.085000,0.000000,85.622,102.000000,132.0,361612.000000,0.532856,0.394685,0.000000
0,2021-11-24 14:00:00,62.865,62.988,62.693,62.764,502837.0,C,5,0.000000,160.308182,...,62.764000,62.764000,0.000000,85.622,0.000000,0.0,0.000000,0.405170,0.686041,-0.925136
0,2021-11-24 14:00:00,158.451,158.655,158.097,158.304,328534.0,JPM,5,0.000000,160.308182,...,158.304000,158.304000,0.000000,85.622,4.000000,3.0,88331.000000,0.427115,0.501671,-0.871115
0,2021-11-24 14:00:00,379.350,379.789,377.857,378.279,78441.0,GS,5,0.000000,160.308182,...,378.279000,378.279000,0.000000,85.622,1.000000,0.0,1.000000,0.376175,0.028926,0.953225
0,2021-11-24 14:00:00,145.744,146.249,145.587,145.825,329860.0,GOOGL,5,0.000000,160.308182,...,145.825000,145.825000,0.000000,85.622,2.000000,2.0,279.000000,0.485056,0.328640,-0.033879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,2023-04-26 10:00:00,59.444,59.938,59.362,59.891,1496473.0,MU,1,-0.559588,61.670013,...,60.116533,60.955850,4705.439114,42.076,0.000000,0.0,0.000000,0.000000,0.316690,-1.131348
2481,2023-04-26 10:00:00,186.561,186.758,185.602,185.652,153780.0,PEP,1,1.081484,188.509769,...,184.072067,182.829733,4705.439114,42.076,1.000000,0.0,35848.666667,0.500000,0.056523,0.802627
2481,2023-04-26 10:00:00,63.061,63.226,62.975,63.101,604614.0,KO,1,-0.025948,63.929023,...,63.422567,63.127017,4705.439114,42.076,1.000000,0.0,10196.000000,0.546800,0.268158,-0.755347


In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})

# Plot Histogram on x
x = filtered
plt.hist(x, bins=20)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');

NameError: name 'filtered' is not defined

In [ ]:
def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [ ]:
filtered = x[~is_outlier(x.values)]

In [6]:
filtered

NameError: name 'filtered' is not defined

In [33]:
df_num = train.select_dtypes(include='number')
df_norm = (df_num - df_num.min()) / (df_num.max() - df_num.min())*200 + 20


train[df_norm.columns] = df_norm

In [34]:
df_norm

,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,...,close_30_sma,close_60_sma,turbulence,vix,stocktwitsPosts,stocktwitsLikes,stocktwitsImpressions,stocktwitsSentiment,random,change
index,,,,,,,,,,,,,,,,,,,,,
0,64.016138,63.988587,64.138571,63.994663,36.200417,186.666667,165.183339,63.770385,64.997297,220.000000,...,64.512513,64.465603,20.000000,161.582472,42.666667,50.877193,47.542370,126.571143,98.931973,118.329497
0,34.827192,34.786757,34.875852,34.787895,22.318159,186.666667,165.183339,63.770385,64.997297,220.000000,...,34.921095,34.865256,20.000000,161.582472,20.000000,20.000000,20.000076,101.033996,157.205789,89.135219
0,63.735181,63.672157,63.905794,63.757846,21.514435,186.666667,165.183339,63.770385,64.997297,220.000000,...,64.272577,64.225594,20.000000,161.582472,20.888889,20.701754,26.727835,105.423009,120.330185,90.839940
0,130.541469,130.440677,130.775316,130.459383,20.361236,186.666667,165.183339,63.770385,64.997297,220.000000,...,131.852568,131.825974,20.000000,161.582472,20.222222,20.000000,20.000152,95.234946,25.776899,148.410179
0,59.892214,59.926327,60.099197,59.973923,21.520549,186.666667,165.183339,63.770385,64.997297,220.000000,...,60.438820,60.390680,20.000000,161.582472,20.444444,20.467836,20.021326,117.011256,85.722364,117.260396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,33.792582,33.865849,33.862281,33.916735,26.899892,53.333333,163.224511,34.201409,33.553319,114.972636,...,34.107749,34.309595,26.939494,89.575858,20.000000,20.000000,20.000076,20.000000,83.332440,82.627833
2481,72.236463,72.157490,72.275134,72.050396,20.708630,53.333333,168.969054,72.224435,71.954183,140.701728,...,72.188957,71.762582,26.939494,89.575858,20.222222,20.000000,22.730501,120.000000,31.296705,143.657790
2481,34.886468,34.858618,34.961660,34.890082,22.787461,53.333333,165.092509,34.878597,35.273524,120.087130,...,35.123417,34.976815,26.939494,89.575858,20.222222,20.000000,20.776658,129.360000,73.625456,94.493198


In [35]:
train

,date,open,high,low,close,volume,tic,day,macd,boll_ub,...,close_30_sma,close_60_sma,turbulence,vix,stocktwitsPosts,stocktwitsLikes,stocktwitsImpressions,stocktwitsSentiment,random,change
index,,,,,,,,,,,,,,,,,,,,,
0,2021-11-24 14:00:00,64.016138,63.988587,64.138571,63.994663,36.200417,AAPL,186.666667,165.183339,63.770385,...,64.512513,64.465603,20.000000,161.582472,42.666667,50.877193,47.542370,126.571143,98.931973,118.329497
0,2021-11-24 14:00:00,34.827192,34.786757,34.875852,34.787895,22.318159,C,186.666667,165.183339,63.770385,...,34.921095,34.865256,20.000000,161.582472,20.000000,20.000000,20.000076,101.033996,157.205789,89.135219
0,2021-11-24 14:00:00,63.735181,63.672157,63.905794,63.757846,21.514435,JPM,186.666667,165.183339,63.770385,...,64.272577,64.225594,20.000000,161.582472,20.888889,20.701754,26.727835,105.423009,120.330185,90.839940
0,2021-11-24 14:00:00,130.541469,130.440677,130.775316,130.459383,20.361236,GS,186.666667,165.183339,63.770385,...,131.852568,131.825974,20.000000,161.582472,20.222222,20.000000,20.000152,95.234946,25.776899,148.410179
0,2021-11-24 14:00:00,59.892214,59.926327,60.099197,59.973923,21.520549,GOOGL,186.666667,165.183339,63.770385,...,60.438820,60.390680,20.000000,161.582472,20.444444,20.467836,20.021326,117.011256,85.722364,117.260396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,2023-04-26 10:00:00,33.792582,33.865849,33.862281,33.916735,26.899892,MU,53.333333,163.224511,34.201409,...,34.107749,34.309595,26.939494,89.575858,20.000000,20.000000,20.000076,20.000000,83.332440,82.627833
2481,2023-04-26 10:00:00,72.236463,72.157490,72.275134,72.050396,20.708630,PEP,53.333333,168.969054,72.224435,...,72.188957,71.762582,26.939494,89.575858,20.222222,20.000000,22.730501,120.000000,31.296705,143.657790
2481,2023-04-26 10:00:00,34.886468,34.858618,34.961660,34.890082,22.787461,KO,53.333333,165.092509,34.878597,...,35.123417,34.976815,26.939494,89.575858,20.222222,20.000000,20.776658,129.360000,73.625456,94.493198


Goal for generated data: one more column for sentiment for each day --> Merge on "tic" and "date"

## Construct the environment

Calculate and specify the parameters we need for constructing the environment.

In [38]:
INDICATORS.append('vix')

INDICATORS = ['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma',
 'vix']

In [40]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma',
 'vix']

In [41]:
SENTIMENT = ['random', 'stocktwitsPosts', 'stocktwitsLikes', 'stocktwitsImpressions', 'stocktwitsSentiment']
#'stocktwitsPosts', 'stocktwitsLikes', 'stocktwitsImpressions', 'stocktwitsSentiment', 

In [42]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 4 * stock_dimension + len(INDICATORS) * stock_dimension + len(SENTIMENT) * stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 30, State Space: 541


In [43]:
seed = 0
hourly = False

In [44]:
buy_cost_list = sell_cost_list = [0.01] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "sentiment_list": SENTIMENT,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "seed": seed,
    "hourly": hourly
}


e_train_gym = StockTradingEnv(df = train,risk_indicator_col='vix', **env_kwargs)

## Environment for training

In [45]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Get onnx model

In [47]:
saved_model_path = r"C:\Users\potap\FinARL\automatic-stock-trading-applied-reinforcement-learning-sommer-term-2023\Training and Testing\Evaluation\model\hourly\Sentiment\ppo\seed0.zip"

In [48]:
agent = DRLAgent(env = env_train)

In [49]:
model_a2c = agent.get_model("ppo")

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cpu device


In [50]:
trained_a2c = agent.train_model(model=model_a2c,
                                tb_log_name='ppo',
                                total_timesteps=500)

-----------------------------------
| time/              |            |
|    fps             | 72         |
|    iterations      | 1          |
|    time_elapsed    | 28         |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | -1.9694179 |
-----------------------------------


In [51]:
loaded_agent = trained_a2c.load(saved_model_path)

In [52]:
# Make sure your model is in evaluation mode
loaded_agent.policy.eval()

# generate dummy input
observation_size = loaded_agent.observation_space.shape[0]
dummy_input = torch.randn(1, observation_size)

# Export the model to ONNX format
torch.onnx.export(loaded_agent.policy, dummy_input, "policy.onnx")


============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================


C:\Users\potap\FinARL\venv_10\lib\site-packages\stable_baselines3\common\distributions.py:87: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if deterministic:


In [53]:
import onnx

from onnx_tf.backend import prepare

onnx_model = onnx.load(r"C:\Users\potap\FinARL\automatic-stock-trading-applied-reinforcement-learning-sommer-term-2023\Generated Sentiment Data in FinRL\policy.onnx")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph(r"C:\Users\potap\FinARL\automatic-stock-trading-applied-reinforcement-learning-sommer-term-2023\Generated Sentiment Data in FinRL\HIERoutput.pb")  # export the model

INFO:tensorflow:Assets written to: C:\Users\potap\FinARL\automatic-stock-trading-applied-reinforcement-learning-sommer-term-2023\Generated Sentiment Data in FinRL\HIERoutput.pb\assets


INFO:tensorflow:Assets written to: C:\Users\potap\FinARL\automatic-stock-trading-applied-reinforcement-learning-sommer-term-2023\Generated Sentiment Data in FinRL\HIERoutput.pb\assets


In [54]:
import tensorflow as tf
model_filename = r"C:\Users\potap\FinARL\automatic-stock-trading-applied-reinforcement-learning-sommer-term-2023\Generated Sentiment Data in FinRL\HIERoutput.pb"
# Load the TensorFlow model
model_loaded = tf.saved_model.load(model_filename)
model_loaded = model_loaded.signatures['serving_default']
model_loaded

<ConcreteFunction signature_wrapper(*, obs) at 0x2CE8D5A4FD0>

#  Sensitivity test

In [55]:
# generate dummy input
#observation_size = model_loaded.observation_space.shape[0]
dummy_input = torch.randn(1, 541)

In [26]:
xs = tf.convert_to_tensor(dummy_input)

In [27]:
with tf.GradientTape() as tape:
    tape.watch(xs)
    pred = model_loaded(xs)
    grads = tape.gradient(pred, xs)

In [ ]:
assert (grads is not None) # sanity check
grads = grads.numpy()
abs_grads_sum = np.sum(np.abs(grads), axis=0)


In [26]:
abs_grads_sum

array([6.141937  , 2.0792143 , 1.1400537 , 2.5549965 , 0.25538453,
       0.1457287 , 0.91045415, 0.96804583, 1.9791678 , 0.5729369 ,
       1.5138545 , 4.8658857 , 2.0542839 , 8.757634  , 2.349398  ,
       0.69924116, 0.51150405, 0.30408013, 2.6819537 , 1.6688392 ,
       0.86045796, 2.3970149 , 4.1402144 , 2.1240876 , 3.4856312 ,
       1.8171442 , 0.95222837, 5.228218  , 2.5927901 , 3.7669134 ,
       4.3272204 , 2.3591924 , 3.966771  , 2.1997342 , 0.16204202,
       2.7498965 , 2.9850717 , 5.7253227 , 1.7466173 , 4.11631   ,
       3.633401  , 0.22664887, 4.170761  , 2.1862285 , 1.8478186 ,
       2.5734751 , 1.0064908 , 1.0574986 , 2.846866  , 0.23595265,
       0.42381045, 2.6337516 , 1.8567915 , 0.72375774, 1.3361348 ,
       1.9963938 , 5.43626   , 1.2364389 , 5.980402  , 5.040382  ,
       0.66498214, 1.7261225 , 2.0128493 , 0.21035725, 1.1941941 ,
       3.3602214 , 0.70596945, 1.2577521 , 2.3316288 , 2.774733  ,
       0.28998908, 0.36586928, 3.451414  , 3.371114  , 2.72316

# Unsere Data

In [56]:
num_stock_shares = 30*[0]

In [57]:
train

,date,open,high,low,close,volume,tic,day,macd,boll_ub,...,close_30_sma,close_60_sma,turbulence,vix,stocktwitsPosts,stocktwitsLikes,stocktwitsImpressions,stocktwitsSentiment,random,change
index,,,,,,,,,,,,,,,,,,,,,
0,2021-11-24 14:00:00,64.016138,63.988587,64.138571,63.994663,36.200417,AAPL,186.666667,165.183339,63.770385,...,64.512513,64.465603,20.000000,161.582472,42.666667,50.877193,47.542370,126.571143,98.931973,118.329497
0,2021-11-24 14:00:00,34.827192,34.786757,34.875852,34.787895,22.318159,C,186.666667,165.183339,63.770385,...,34.921095,34.865256,20.000000,161.582472,20.000000,20.000000,20.000076,101.033996,157.205789,89.135219
0,2021-11-24 14:00:00,63.735181,63.672157,63.905794,63.757846,21.514435,JPM,186.666667,165.183339,63.770385,...,64.272577,64.225594,20.000000,161.582472,20.888889,20.701754,26.727835,105.423009,120.330185,90.839940
0,2021-11-24 14:00:00,130.541469,130.440677,130.775316,130.459383,20.361236,GS,186.666667,165.183339,63.770385,...,131.852568,131.825974,20.000000,161.582472,20.222222,20.000000,20.000152,95.234946,25.776899,148.410179
0,2021-11-24 14:00:00,59.892214,59.926327,60.099197,59.973923,21.520549,GOOGL,186.666667,165.183339,63.770385,...,60.438820,60.390680,20.000000,161.582472,20.444444,20.467836,20.021326,117.011256,85.722364,117.260396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481,2023-04-26 10:00:00,33.792582,33.865849,33.862281,33.916735,26.899892,MU,53.333333,163.224511,34.201409,...,34.107749,34.309595,26.939494,89.575858,20.000000,20.000000,20.000076,20.000000,83.332440,82.627833
2481,2023-04-26 10:00:00,72.236463,72.157490,72.275134,72.050396,20.708630,PEP,53.333333,168.969054,72.224435,...,72.188957,71.762582,26.939494,89.575858,20.222222,20.000000,22.730501,120.000000,31.296705,143.657790
2481,2023-04-26 10:00:00,34.886468,34.858618,34.961660,34.890082,22.787461,KO,53.333333,165.092509,34.878597,...,35.123417,34.976815,26.939494,89.575858,20.222222,20.000000,20.776658,129.360000,73.625456,94.493198


In [59]:
all_days_grads = []
for day in range(len((train.date.unique()))): # len((train.date.unique()))
   data = train.loc[day, :]
   
   state = (
    [50]
    + data.close.values.tolist()
    + num_stock_shares
    + sum(
        (
            data[tech].values.tolist()
            for tech in INDICATORS
        ),
        [],
    )
    + sum(
    (
        data[sent].values.tolist()
        for sent in SENTIMENT
    ),
    [],
)
    + data.change.values.tolist()
    + data.volume.values.tolist())
   
   #print(state)
   xs = tf.convert_to_tensor([state])
   with tf.GradientTape() as tape:
        tape.watch(xs)
        pred = model_loaded(xs)
        grads = tape.gradient(pred, xs)
   grads = grads.numpy()
   abs_grads_sum = np.sum(np.abs(grads), axis=0)
   all_days_grads.append(abs_grads_sum)

In [ ]:
all_days_grads

In [32]:
import matplotlib
matplotlib.use('TkAgg')  # or another appropriate backend like 'Qt5Agg'
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
# Create a heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(all_days_grads, cmap="YlGnBu", cbar=True)

# Display the plot
plt.title('Heatmap of all_days_grads')
plt.xlabel('Index within each sublist')
plt.ylabel('List index')
plt.show()

NameError: name 'plt' is not defined

In [48]:
len(state)

421

In [53]:
sums = [sum(day[i] for day in all_days_grads) for i in range(421)]
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))
plt.bar(range(421), sums)
plt.title('Sum of values at each index across all days')
plt.xlabel('Index')
plt.ylabel('Sum')
plt.show()
